# 보행자 학습을 위한 전처리 과정 (aihub data)

### 1. 부천 남부역 주차장

In [1]:
import json
from glob import glob
import os
from itertools import compress

In [2]:
json_path = 'C:/Users/82103/OneDrive/바탕 화면/교통문제 해결을 위한 CCTV 교통 영상(시내도로)/Training/교통안전(Bbox)/[라벨]교통안전(Bbox)/부천남부역 공영주차장/BC2000103/부천남부역공영주차장_BC2000103.json'

In [72]:
# json file 불러오기

with open(json_path, "r", encoding = "UTF-8") as json_file:
    json_data = json.load(json_file)

In [74]:
# annotation 내 존재하는 label number 와 name 
json_data['categories']

[{'id': 1, 'name': '승용차'},
 {'id': 2, 'name': '소형버스'},
 {'id': 3, 'name': '대형버스'},
 {'id': 4, 'name': '트럭'},
 {'id': 5, 'name': '대형 트레일러'},
 {'id': 6, 'name': '오토바이(자전거)'},
 {'id': 7, 'name': '보행자'},
 {'id': 8, 'name': '분류없음'}]

In [75]:
# bounding box 위치 좌표 변환
# 현재 데이터 셋 COCO 좌표 이므로, yolo 형식에 맞게 변환해줘야함

def bbox(anno_info):
    bbox = []

    for m_bbox,cate_id in zip(anno_info['bbox'],anno_info['category_id']):
        xyxy=m_bbox

        b_center_x = (xyxy[0] + xyxy[2]) / 2
        b_center_y = (xyxy[1] + xyxy[3]) / 2
        b_width    = xyxy[2] - xyxy[0]
        b_height   = xyxy[3] - xyxy[1]

        b_center_x/=1920
        b_center_y/=1080
        b_width/=1920
        b_height/=1080
        cateid = cate_id -1 # 1-8 로 되어있어서 -1 해줘야함 (yolo는 class 를 0부터 읽음)

        bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(cateid, b_center_x, b_center_y, b_width, b_height))

    return bbox

In [36]:
# 각 프레임 이름으로 anntation label 정보 저장

save_path = 'E:/train_person/labels/'   

def save_anno(json_data):
    image = json_data['images']
    anno = json_data['annotations']
    
    for i in range(len(image)):
        name = image[i]['file_name'].split("/")[-1].split(".")[0]
        
        bbox_t = bbox(anno[i])
        fil = [x[0] == '6' for x in bbox_t]
        anno_p = list(compress(bbox_t, fil))
        anno_p = ['0' + x[1:] for x in anno_p] # label 0 으로 변환해서 person 으로 따로 학습시켜보기 위함  
        
        if len(anno_p) != 0:
            with open(save_path + name + '.txt', "w", encoding = "UTF-8") as f:
                f.write('\n'.join(anno_p) + '\n')
            print(f'{i} th in person!')

## 사진 프레임 낮추기

In [15]:
import cv2
import time
import imutils
from glob import glob
from PIL import Image
import os

# output 경로 지정
output_path = 'E:/train_person/images/' 
# 불러올 이미지 파일 경로 지정
load_path = 'C:/Users/82103/OneDrive/바탕 화면/교통문제 해결을 위한 CCTV 교통 영상(시내도로)/Training/교통안전(Bbox)/부천남부역 공영주차장/BC2000103/'
# 위에서 뽑아낸 label 파일 경로
label_path = 'E:/train_person/labels/'
# 파일명 리스트 불러와서
file = os.listdir(label_path)

# 이름만 가져와서 jpg 파일 붙인 후, load_path 에서 불러오기
file_list = [x.split(".")[0] + ".jpg" for x in file]

def save_img(file_list):
    
    for i in range(len(file_list)):
        cap = cv2.VideoCapture(load_path + file_list[i])
        hasFrame, img = cap.read()
        img_frame = imutils.resize(img, width=640, height = 360)
        cap.release()
        cv2.imwrite(output_path + file_list[i], img_frame)
        print(f'{i} th done!')

### 2. 보강센트럴빌 2차 앞

In [41]:
json_path = 'C:/Users/82103/OneDrive/바탕 화면/교통문제 해결을 위한 CCTV 교통 영상(시내도로)/Training/교통안전(Bbox)/[라벨]교통안전(Bbox)/보강센트럴빌 2차 앞/BC2000203/보강센트럴빌 2차 앞_BC2000203.json'

In [42]:
with open(json_path, "r", encoding = "UTF-8") as json_file:
    json_data = json.load(json_file)

In [43]:
json_data.keys()

dict_keys(['info', 'images', 'annotations', 'categories', 'guideline', 'meta'])

In [44]:
json_data['categories']

[{'id': 1, 'name': '승용차'},
 {'id': 2, 'name': '소형버스'},
 {'id': 3, 'name': '대형버스'},
 {'id': 4, 'name': '트럭'},
 {'id': 5, 'name': '대형 트레일러'},
 {'id': 6, 'name': '오토바이(자전거)'},
 {'id': 7, 'name': '보행자'},
 {'id': 8, 'name': '분류없음'}]

In [45]:
save_path = 'E:/train_person/labels/wow/'   

def save_anno(json_data):
    image = json_data['images']
    anno = json_data['annotations']
    
    for i in range(len(image)):
        name = image[i]['file_name'].split("/")[-1].split(".")[0]
        
        bbox_t = bbox(anno[i])
        fil = [x[0] == '6' for x in bbox_t]
        anno_p = list(compress(bbox_t, fil))
        anno_p = ['0' + x[1:] for x in anno_p] # label 0 으로 변환 
        
        if len(anno_p) != 0:
            with open(save_path + name + '.txt', "w", encoding = "UTF-8") as f:
                f.write('\n'.join(anno_p) + '\n')
            print(f'{i} th in person!')

In [47]:
import cv2
import time
import imutils
from glob import glob
from PIL import Image
import os

output_path = 'E:/train_person/images/'
load_path = 'C:/Users/82103/OneDrive/바탕 화면/교통문제 해결을 위한 CCTV 교통 영상(시내도로)/Training/교통안전(Bbox)/보강센트럴빌 2차 앞/BC2000203/'
label_path = 'E:/train_person/labels/wow/' 
file = os.listdir(label_path)
file_list = [x.split(".")[0] + ".jpg" for x in file]

save_img(file_list)



0 th done!
1 th done!
2 th done!
3 th done!
4 th done!
5 th done!
6 th done!
7 th done!
8 th done!
9 th done!
10 th done!
11 th done!
12 th done!
13 th done!
14 th done!
15 th done!
16 th done!
17 th done!
18 th done!
19 th done!
20 th done!
21 th done!
22 th done!
23 th done!
24 th done!
25 th done!
26 th done!
27 th done!
28 th done!
29 th done!
30 th done!
31 th done!
32 th done!
33 th done!
34 th done!
35 th done!
36 th done!
37 th done!
38 th done!
39 th done!
40 th done!
41 th done!
42 th done!
43 th done!
44 th done!
45 th done!
46 th done!
47 th done!
48 th done!
49 th done!
50 th done!
51 th done!
52 th done!
53 th done!
54 th done!
55 th done!
56 th done!
57 th done!
58 th done!
59 th done!
60 th done!
61 th done!
62 th done!
63 th done!
64 th done!
65 th done!
66 th done!
67 th done!
68 th done!
69 th done!
70 th done!
71 th done!
72 th done!
73 th done!
74 th done!
75 th done!
76 th done!
77 th done!
78 th done!
79 th done!
80 th done!
81 th done!
82 th done!
83 th done!
84

# 2. 1인칭 주행 시점

### 2-1 빌딩 주변

In [45]:
from glob import glob

In [46]:
path = 'E:/train_images/1인칭 시점 보행영상/Training/BBOX/Average_stature/Building_area/'

In [47]:
# 하위 폴더까지 다 불러오기
output = glob(path + '**', recursive=True)
output = [x.split('\\')[-2] + "/" + x.split('\\')[-1] for x in output]

In [48]:
# json 다 불러오기
json_list = [x for x in output if x.split(".")[-1] == 'json']

In [50]:
# 이미지도 다 불러오기
file_list = [x for x in output if x.split(".")[-1] == 'png']

In [52]:
with open(path + json_list[0], "r", encoding = "UTF-8") as json_file:
    json_data = json.load(json_file)

In [53]:
# 좌표 버전이 다르기 때문에 다른 거 적용

def bbox_2(anno_info):
    
    bbox = []

    for i in range(len(anno_info)):
        
        xyxy=list(anno_info[i]["box"].values())

        b_center_x = xyxy[0] + xyxy[2]/2
        b_center_y = xyxy[1] + xyxy[3]/2
        b_width    = xyxy[2]
        b_height   = xyxy[3]

        b_center_x/=1920
        b_center_y/=1080
        b_width/=1920
        b_height/=1080

        bbox.append("{} {:.6f} {:.6f} {:.6f} {:.6f}".format(0, b_center_x, b_center_y, b_width, b_height))

    
    return(bbox)

In [54]:
save_path = 'E:/train_images/labels/wow/'   

def save_anno(json_data):
    
    anno = json_data['annotation']['annotations']
    video_name = json_data['annotation']['resource'].split(".")[0]
    
    for i in range(len(anno)):
        fps_name = anno[i]['atchOrgFileName'].split(".")[0]
        name = video_name + "__" + fps_name
        
        bbox = anno[i]['box']
        bbox = [x for x in bbox if x['category_name'] == 'person']
        anno_p = bbox_2(bbox)
        
        if len(anno_p) != 0:
            with open(save_path + name + '.txt', "w", encoding = "UTF-8") as f:
                f.write('\n'.join(anno_p) + '\n')
            print(f'{i} th in person!')

In [55]:
path = 'E:/train_images/1인칭 시점 보행영상/Training/BBOX/Average_stature/Building_area/'

def save_anno_list(json_list):
    for j in range(len(json_list)):
        with open(path + json_list[j], "r", encoding = "UTF-8") as json_file:
            json_data = json.load(json_file)
        save_anno(json_data)
        print(f'{j} th json file done!!!!!!!!!!!!!!')

In [56]:
save_anno_list(json_list)

0 th json file done!!!!!!!!!!!!!!
20 th in person!
21 th in person!
22 th in person!
23 th in person!
24 th in person!
25 th in person!
26 th in person!
38 th in person!
39 th in person!
40 th in person!
41 th in person!
42 th in person!
43 th in person!
54 th in person!
1 th json file done!!!!!!!!!!!!!!
0 th in person!
1 th in person!
2 th in person!
3 th in person!
5 th in person!
11 th in person!
12 th in person!
13 th in person!
14 th in person!
15 th in person!
16 th in person!
17 th in person!
18 th in person!
19 th in person!
20 th in person!
21 th in person!
22 th in person!
23 th in person!
24 th in person!
25 th in person!
26 th in person!
27 th in person!
28 th in person!
29 th in person!
30 th in person!
31 th in person!
34 th in person!
35 th in person!
36 th in person!
37 th in person!
38 th in person!
40 th in person!
41 th in person!
46 th in person!
47 th in person!
48 th in person!
49 th in person!
50 th in person!
51 th in person!
52 th in person!
2 th json file done

In [57]:
# 한글 경로 해결

import cv2
import numpy as np
import re
import tempfile
import shutil
def imreadEX(image_path):
    if re.compile('[^ㄱ-ㅣ가-힣]+').sub('', image_path):
            stream = open(image_path, "rb")
            bytes = bytearray(stream.read())
            numpyarray = np.asarray(bytes, dtype=np.uint8)
            img = cv2.imdecode(numpyarray, cv2.IMREAD_UNCHANGED)
            if not img is None:
                return img
            else:
                file_tmp=tempfile.NamedTemporaryFile().name
                shutil.copy(image_path,file_tmp)
                image_path=file_tmp
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    return img

In [58]:
import cv2
import time
import imutils
from glob import glob
from PIL import Image
import os

output_path = 'E:/train_images/images/'
load_path = 'E:/train_images/1인칭 시점 보행영상/Training/BBOX/Average_stature/Building_area/'
label_path = 'E:/train_images/labels/wow/' 
 
file = os.listdir(label_path)
img_file = [x.split(".")[0] + ".png" for x in file] # 사람이 있는 img list

output = glob(path + '**', recursive=True)
output = [x.split('\\')[-2] + "/" + x.split('\\')[-1] for x in output] # 각각의 경로와 파일 이름
file_list = [x for x in output if x.split(".")[-1] == 'png']

img_list = [x for x in file_list if x.split('/')[1] in img_file] # 최종 경로와 사람 파일

def save_img(file_list):
    
    for i in range(len(file_list)):
        img = imreadEX(load_path + img_list[i])
        img_frame = imutils.resize(img, width=640, height = 360)
        cv2.imwrite(output_path + file_list[i].split('/')[1].split(".")[0] + '.jpg', img_frame)
        print(f'{i} th done!')

In [60]:
save_img(img_list)

0 th done!
1 th done!
2 th done!
3 th done!
4 th done!
5 th done!
6 th done!
7 th done!
8 th done!
9 th done!
10 th done!
11 th done!
12 th done!
13 th done!
14 th done!
15 th done!
16 th done!
17 th done!
18 th done!
19 th done!
20 th done!
21 th done!
22 th done!
23 th done!
24 th done!
25 th done!
26 th done!
27 th done!
28 th done!
29 th done!
30 th done!
31 th done!
32 th done!
33 th done!
34 th done!
35 th done!
36 th done!
37 th done!
38 th done!
39 th done!
40 th done!
41 th done!
42 th done!
43 th done!
44 th done!
45 th done!
46 th done!
47 th done!
48 th done!
49 th done!
50 th done!
51 th done!
52 th done!
53 th done!
54 th done!
55 th done!
56 th done!
57 th done!
58 th done!
59 th done!
60 th done!
61 th done!
62 th done!
63 th done!
64 th done!
65 th done!
66 th done!
67 th done!
68 th done!
69 th done!
70 th done!
71 th done!
72 th done!
73 th done!
74 th done!
75 th done!
76 th done!
77 th done!
78 th done!
79 th done!
80 th done!
81 th done!
82 th done!
83 th done!
84

### 2-2 거주지 주변

In [61]:
path = 'E:/train_images/1인칭 시점 보행영상/Training/BBOX/Mobility_stature/Residential_area_3/'

In [62]:
# 하위 폴더까지 다 불러오기
output = glob(path + '**', recursive=True)
output = [x.split('\\')[-2] + "/" + x.split('\\')[-1] for x in output]

In [64]:
# json 다 불러오기
json_list = [x for x in output if x.split(".")[-1] == 'json']

# 이미지도 다 불러오기
file_list = [x for x in output if x.split(".")[-1] == 'png']

In [65]:
with open(path + json_list[0], "r", encoding = "UTF-8") as json_file:
    json_data = json.load(json_file)

In [66]:
save_path = 'E:/train_images/labels/wow/'   

def save_anno(json_data):
    
    anno = json_data['annotation']['annotations']
    video_name = json_data['annotation']['resource'].split(".")[0]
    
    for i in range(len(anno)):
        fps_name = anno[i]['atchOrgFileName'].split(".")[0]
        name = video_name + "__" + fps_name
        
        bbox = anno[i]['box']
        bbox = [x for x in bbox if x['category_name'] == 'person']
        anno_p = bbox_2(bbox)
        
        if len(anno_p) != 0:
            with open(save_path + name + '.txt', "w", encoding = "UTF-8") as f:
                f.write('\n'.join(anno_p) + '\n')
            print(f'{i} th in person!')
            
def save_anno_list(json_list):
    for j in range(len(json_list)):
        with open(path + json_list[j], "r", encoding = "UTF-8") as json_file:
            json_data = json.load(json_file)
        save_anno(json_data)
        print(f'{j} th json file done!!!!!!!!!!!!!!')

In [67]:
save_anno_list(json_list)

7 th in person!
8 th in person!
9 th in person!
10 th in person!
20 th in person!
0 th json file done!!!!!!!!!!!!!!
22 th in person!
1 th json file done!!!!!!!!!!!!!!
2 th json file done!!!!!!!!!!!!!!
4 th in person!
5 th in person!
7 th in person!
9 th in person!
11 th in person!
19 th in person!
20 th in person!
21 th in person!
22 th in person!
23 th in person!
24 th in person!
25 th in person!
26 th in person!
27 th in person!
29 th in person!
30 th in person!
31 th in person!
3 th json file done!!!!!!!!!!!!!!
4 th json file done!!!!!!!!!!!!!!
0 th in person!
1 th in person!
2 th in person!
3 th in person!
4 th in person!
5 th in person!
6 th in person!
7 th in person!
8 th in person!
9 th in person!
10 th in person!
19 th in person!
20 th in person!
21 th in person!
22 th in person!
23 th in person!
24 th in person!
25 th in person!
26 th in person!
27 th in person!
28 th in person!
29 th in person!
30 th in person!
31 th in person!
5 th json file done!!!!!!!!!!!!!!
0 th in person

In [68]:
import cv2
import time
import imutils
from glob import glob
from PIL import Image
import os

output_path = 'E:/train_images/images/'
load_path = 'E:/train_images/1인칭 시점 보행영상/Training/BBOX/Mobility_stature/Residential_area_3/'
label_path = 'E:/train_images/labels/wow/' 
 
file = os.listdir(label_path)
img_file = [x.split(".")[0] + ".png" for x in file] # 사람이 있는 img list

output = glob(path + '**', recursive=True)
output = [x.split('\\')[-2] + "/" + x.split('\\')[-1] for x in output] # 각각의 경로와 파일 이름
file_list = [x for x in output if x.split(".")[-1] == 'png']

img_list = [x for x in file_list if x.split('/')[1] in img_file] # 최종 경로와 사람 파일

def save_img(file_list):
    
    for i in range(len(file_list)):
        img = imreadEX(load_path + img_list[i])
        img_frame = imutils.resize(img, width=640, height = 360)
        cv2.imwrite(output_path + file_list[i].split('/')[1].split(".")[0] + '.jpg', img_frame)
        print(f'{i} th done!')

In [69]:
save_img(img_list)

0 th done!
1 th done!
2 th done!
3 th done!
4 th done!
5 th done!
6 th done!
7 th done!
8 th done!
9 th done!
10 th done!
11 th done!
12 th done!
13 th done!
14 th done!
15 th done!
16 th done!
17 th done!
18 th done!
19 th done!
20 th done!
21 th done!
22 th done!
23 th done!
24 th done!
25 th done!
26 th done!
27 th done!
28 th done!
29 th done!
30 th done!
31 th done!
32 th done!
33 th done!
34 th done!
35 th done!
36 th done!
37 th done!
38 th done!
39 th done!
40 th done!
41 th done!
42 th done!
43 th done!
44 th done!
45 th done!
46 th done!
47 th done!
48 th done!
49 th done!
50 th done!
51 th done!
52 th done!
53 th done!
54 th done!
55 th done!
56 th done!
57 th done!
58 th done!
59 th done!
60 th done!
61 th done!
62 th done!
63 th done!
64 th done!
65 th done!
66 th done!
67 th done!
68 th done!
69 th done!
70 th done!
71 th done!
72 th done!
73 th done!
74 th done!
75 th done!
76 th done!
77 th done!
78 th done!
79 th done!
80 th done!
81 th done!
82 th done!
83 th done!
84